In [ ]:
#Model 2 with required results
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.data import Dataset
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Load dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Function to preprocess and resize images
def preprocess_images(images):
    images = tf.expand_dims(images, -1)  # Add a channel dimension
    images = tf.repeat(images, 3, axis=-1)  # Convert to RGB
    images = tf.image.resize_with_pad(images, 96, 96)  # Resize images
    return images / 255.0  # Normalize the images

# Convert the datasets to TensorFlow Dataset objects
def get_dataset(x, y, batch_size=32):
    dataset = Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(lambda img, lbl: (preprocess_images(img), lbl), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

train_dataset = get_dataset(train_images, train_labels)
test_dataset = get_dataset(test_images, test_labels)

# Load a pre-trained MobileNetV2 model as a feature extractor
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
base_model.trainable = False

# Attach a new classification head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model using the dataset
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test Accuracy: {test_acc}")

# Predict the test dataset
y_pred = model.predict(test_dataset)
y_pred_classes = np.argmax(y_pred, axis=1)

# Confusion matrix
conf_matrix = confusion_matrix(test_labels, y_pred_classes)
print(conf_matrix)

# Classification report
print(classification_report(test_labels, y_pred_classes))

# Plotting training and validation accuracy
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

# Plotting training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Plotting the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap='Blues')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()
